In [1]:
import pydriller
pydriller.__version__

'2.8'

In [2]:
import radon
radon.__version__

'6.0.1'

In [1]:
import pandas as pd
pd.read_csv('Final_rectification.csv')

,Unnamed: 0,Hash,Overall_diff,Message,Filename,New Rectified Message
0,0,4b97a810b509c93f44be4c037c7aa18fb8922884,"@@ -1,23 +1,25 @@\n <div class=""align-center"">...","Pre-release 2023 December version (Mistral, Pr...","['README.md', 'Discord.png', 'LAION 2GPU.png',...","Pre-release 2023 December version (Mistral, Pr..."
1,1,2d5d88487463e76f75002be3b2704267ec96e68a,"@@ -19,6 +19,14 @@ import warnings\n import gc...",tokenizer pad fix,"['_utils.py', 'llama.py', 'mistral.py']",Fix tokenizer pad token issue and add new util...
2,2,f380cc1170447800c112dc8568bdff3dd34c79a3,"@@ -7,7 +7,7 @@\n ## 2-5x faster 60% less memo...",Fix Mistral\n\nBlockDiagonalCausalMask fix cou...,"['README.md', 'mistral.py']",Fix Mistral attention mask: replace deprecated...
3,3,de855c2afa68ada67d8b3caad4eb9bb592bf4a4f,"@@ -33,7 +33,7 @@ If you trained a model with ...",Small fixes (#48)\n\n* Fix generation for GQA\...,"['README.md', 'unsloth made with love.png', 'u...",Small fixes (#48)\n\n* Fix generation for GQA ...
4,4,51dd120e354cd2223df7ebe2240fb6d1a76108c5,"@@ -369,6 +369,7 @@ def LlamaModel_fast_forwar...",Fix RoPE Scaling issues (#52)\n\n* Fix RoPE Sc...,"['llama.py', 'mistral.py']",Fix RoPE Scaling issues by updating max_positi...
...,...,...,...,...,...,...
341,341,26601f9d42b4c416efa59a062665c858b94c8673,"@@ -37,7 +37,7 @@ triton = [\n ]\n \n huggingf...",Bug fixes (#3195)\n\n* Fix mamba\n\n* Update l...,"['pyproject.toml', '__init__.py', 'import_fixe...",Bug fixes:\n\n* Update unsloth_zoo version fro...
342,342,e45ddb55a7193be9df23bd72f7396849a0089b2b,"@@ -185,7 +185,15 @@ def Qwen3Attention_fast_f...",fix is casual for qwen3 (#3213),['qwen3.py'],fix is_causal logic for Qwen3: add proper caus...
343,343,b753ec05c1ae49ab2fedc0e252f73c829e36b442,"@@ -149,9 +149,6 @@ class FastLanguageModel(Fa...",GPT OSS Bug fixes (#3231)\n\n* Update rl.py\n\...,"['loader.py', 'rl.py']",Fix gradient checkpointing and data collator p...
344,344,9050d6f5278e2373e66d17fc9ac8c2ec4f986e7e,"@@ -0,0 +1,18 @@\n+#!/bin/bash\n+set -e\n+\n+e...",tests for mxfp4 and quantized models merge fix...,"['run_test.sh', 'test_merged_model.py', 'train...",Add training and inference scripts with proper...


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "unsloth/Qwen3-4B-Instruct-2507-unsloth-bnb-4bit"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load model in 4-bit (bnb) without unsloth
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",  
    torch_dtype=torch.float16
)

In [21]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load tokenizer (CPU is fine for this)
tokenizer = AutoTokenizer.from_pretrained("mamiksik/CommitPredictorT5")

# Load model and move it to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

model = AutoModelForSeq2SeqLM.from_pretrained("mamiksik/CommitPredictorT5",device_map="auto")

Using device: cpu


In [ ]:
df =1

In [ ]:
def predict_batch():
    pass

In [ ]:
import pandas as pd
from tqdm import tqdm

BATCH_SIZE = 10

for i in tqdm(range(5, len(df), BATCH_SIZE), desc="Processing batches"):
    batch_msgs = df["Message"].iloc[i:i+BATCH_SIZE].tolist()
    batch_diffs = df["Overall_diff"].iloc[i:i+BATCH_SIZE].tolist()

    preds = predict_batch(model, tokenizer, batch_msgs, batch_diffs, device="cuda")

    df.loc[i:i+len(preds)-1, "Rectified Message"] = preds

In [22]:
import torch, gc, time

def predict(model, input_text, device="cuda"):
    # Tokenize (truncate to 512 tokens)
    inputs = tokenizer(
        input_text,
        return_tensors="pt",
        truncation=True,
        max_length=512
    ).to(device)

    start_time = time.time()
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            num_beams=5,
            do_sample=True,
            top_p=0.9,
            max_length=64,
            early_stopping=True
        )

    end_time = time.time()
    print("Time taken:", end_time - start_time, "seconds.")

    # Move to CPU before decoding
    prediction = tokenizer.decode(outputs[0].cpu(), skip_special_tokens=True)

    # Free memory
    del inputs, outputs
    torch.cuda.empty_cache()
    gc.collect()

    return prediction

In [4]:
repo_path = "https://github.com/unslothai/unsloth"
L=[]
for commit_hash in concat_df['Hash'].to_list():
    commit_diff = get_commit_diff(repo_path, commit_hash)
    L.append(commit_diff)